In [1]:
import json
with open('data-example/tmp.json') as f:
    raw_data = json.load(f)

In [2]:
X = raw_data[1]
Y = [raw_data[0][i][4:8] for i in range(len(X))]
PX = [raw_data[0][i][0]+raw_data[0][i][2] for i in range(len(X))]
PY = [raw_data[0][i][1]+raw_data[0][i][3] for i in range(len(X))]

In [3]:
X = [X, X, X]  # [3 * 448 * 65]
Y = [Y, Y, Y]  # [3 * 448 * 4]
PX = [PX, PX, PX]  # [3 * 448]
PY = [PY, PY, PY]

In [4]:
music_len = 448
print len(X), len(X[0]), len(X[0][0])
print len(Y), len(Y[0]), len(Y[0][0])
print len(PX), len(PX[0])

3 448 65
3 448 4
3 448


In [5]:
batch_size = 1
num_hidden = 200
X_len = 65
Y_len = 4

no_of_batches = int(len(X) / batch_size)
epoch = 10

In [6]:
import tensorflow as tf
data = tf.placeholder(tf.float32, [None, music_len, X_len])
target = tf.placeholder(tf.float32, [None, music_len, Y_len])
target_PX = tf.placeholder(tf.float32, [None, music_len])
target_PY = tf.placeholder(tf.float32, [None, music_len])

dfm = {data:X, target:Y, target_PX:PX, target_PY:PY}

cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
output, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
# output: [batch, length, feature]

In [7]:
weight = tf.Variable(tf.truncated_normal([num_hidden, Y_len]))
bias = tf.Variable(tf.constant(0.0, shape=[Y_len]))

weight_PX = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias_PX = tf.Variable(tf.constant(0.5, shape=[1]))

weight_PY = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias_PY = tf.Variable(tf.constant(0.5, shape=[1]))

In [8]:
val = tf.reshape(output, [-1, num_hidden])
tar = tf.reshape(target, [-1, Y_len])
tar_PX = tf.reshape(target_PX, [-1])
tar_PY = tf.reshape(target_PY, [-1])

preds = tf.matmul(val, weight) + bias
preds_PX = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PX) + bias_PX, [-1]))
preds_PY = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PY) + bias_PY, [-1]))

In [9]:
cross_ens = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(preds, tar))
l2_PX = tf.reduce_mean(tf.nn.l2_loss(preds_PX - tar_PX))
l2_PY = tf.reduce_mean(tf.nn.l2_loss(preds_PY - tar_PY))

losses = 100 * cross_ens + l2_PX + l2_PY

In [10]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(losses)

In [11]:
mistakes = tf.not_equal(tf.argmax(tar, 1), tf.argmax(preds, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [22]:
print sess.run(losses, dfm)

303.681


In [35]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [37]:
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, out_PX, out_PY = X[ptr:ptr+batch_size], Y[ptr:ptr+batch_size], PX[ptr:ptr+batch_size], PY[ptr:ptr+batch_size]
        ptr += batch_size
        sess.run(minimize, {data: inp, target: out, target_PX: out_PX, target_PY: out_PY})
    incor = sess.run(error, dfm)
    loss = sess.run(losses, dfm)
    cross = sess.run(cross_ens, dfm)
    lx = sess.run(l2_PX, dfm)
    ly = sess.run(l2_PY, dfm)
    print('Epoch {:2d} loss {:3.6f}'.format(i + 1, loss))
#     print('         error {:3.6f}% cross {:3.6f} lx {:3.6f} ly {:3.6f}'.format(incor*100, cross, lx, ly))

Epoch  1 loss 80.695610
Epoch  2 loss 76.433853
Epoch  3 loss 72.105530
Epoch  4 loss 66.979706
Epoch  5 loss 62.400642
Epoch  6 loss 58.457840
Epoch  7 loss 55.449379
Epoch  8 loss 52.921150
Epoch  9 loss 50.323631
Epoch 10 loss 47.787582
Epoch 11 loss 45.712944
Epoch 12 loss 44.207130
Epoch 13 loss 42.386452
Epoch 14 loss 40.818974
Epoch 15 loss 38.923786
Epoch 16 loss 36.910561
Epoch 17 loss 35.205547
Epoch 18 loss 34.062664
Epoch 19 loss 32.910248
Epoch 20 loss 31.846874
Epoch 21 loss 30.813692
Epoch 22 loss 29.568481
Epoch 23 loss 28.451324
Epoch 24 loss 27.091072
Epoch 25 loss 26.276802
Epoch 26 loss 25.756454
Epoch 27 loss 24.890463
Epoch 28 loss 23.834518
Epoch 29 loss 22.988739
Epoch 30 loss 22.639149
Epoch 31 loss 22.185116
Epoch 32 loss 21.564281
Epoch 33 loss 21.267155
Epoch 34 loss 20.987957
Epoch 35 loss 20.720852
Epoch 36 loss 20.262810
Epoch 37 loss 19.990379
Epoch 38 loss 19.676365
Epoch 39 loss 19.221210
Epoch 40 loss 18.322138
Epoch 41 loss 17.734787
Epoch 42 loss 17

In [34]:
sess.close()